In [22]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def solve_sdp_with_gurobi(M_g1, M_g2, M_I):
    num_rows = M_g1.shape[0]
    m = gp.Model("sdp_with_gurobi")

    mu_1 = m.addVar(lb=0.0, name="mu_1")
    mu_2 = m.addVar(lb=-GRB.INFINITY, name="mu_2")

    # 行列変数
    lmi_matrix = m.addMVar((num_rows, num_rows),
                           name="LMI_matrix",
                           vtype=GRB.CONTINUOUS)

    # (1) 等式制約
    m.addConstr(lmi_matrix == M_g2 + mu_1 * M_g1 - mu_2 * M_I)

    # (2) PSD 制約 （半正定値制約）
    m.addConstr(lmi_matrix, GRB.SDP, name="sdp_constr")

    # 目的
    m.setObjective(mu_2, GRB.MAXIMIZE)
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print(f"Optimal mu_1: {mu_1.X:.4f}")
        max_revenue = -mu_2.X
        print(f"Maximum Revenue (from mu_2): {max_revenue:.4f}")
        return mu_1.X, mu_2.X, m.Runtime
    else:
        print("Optimization was not successful.")
        return None, None


In [23]:
import mosek

In [24]:
import numpy as np
from numpy.typing import NDArray
from typing import Tuple

def create_sdp_matrices(
    intercepts: NDArray,
    coefficients: NDArray,
    p_bar: NDArray,
    epsilon: float,
) -> Tuple[NDArray, NDArray]:
    """
    価格最適化のSDP定式化に必要な行列 M(g1) と M(g2) を作成します。

    Args:
        intercepts (NDArray): 需要予測モデルの切片 (θ₀)
        coefficients (NDArray): 需要予測モデルの係数 (Θ₀)
        p_bar (NDArray): 平均価格ベクトル (p̄)
        epsilon (float): 許容される誤差 (ε)

    Returns:
        Tuple[NDArray, NDArray]: (M(g1), M(g2))
    """
    num_products = len(intercepts)
    matrix_size = num_products + 1

    # ---- 制約関数 g₁(p) の行列表現 M(g1) ----
    # g₁(p) = pᵀp - 2p̄ᵀp + (p̄ᵀp̄ - ε)
    m_g1 = np.zeros((matrix_size, matrix_size))
    m_g1[0, 0] = p_bar @ p_bar - epsilon  # 定数項 c
    m_g1[0, 1:] = -p_bar.T               # 線形項 b/2
    m_g1[1:, 0] = -p_bar                 # 線形項 b/2
    m_g1[1:, 1:] = np.identity(num_products) # 二次項 A

    # ---- 目的関数 g₂(p) の行列表現 M(g2) ----
    # g₂(p) = -pᵀΘ₀p - θ₀ᵀp
    m_g2 = np.zeros((matrix_size, matrix_size))
    # 定数項 c は 0
    m_g2[0, 1:] = -0.5 * intercepts      # 線形項 b/2
    m_g2[1:, 0] = -0.5 * intercepts      # 線形項 b/2
    m_g2[1:, 1:] = -coefficients         # 二次項 A

    return m_g1, m_g2

In [8]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

from src.create_data.create_data import create_data
from src.linear_regression.build_model import build_model
from src.ex_price_optimize.ex_optimize_price import optimize_price
from src.new_price_optimize.new_optimize_price import optimize_price_new,optimize_price_new_mosek
from src.new_price_optimize.new_optimize_price_bdf import optimize_price_new_bdf
from src.new_price_optimize.new_optimize_price_sdp import optimize_price_sdp_mosek
from src.evalut_price.evalut_optimal_prices import calculate_true_revenue

In [9]:
alpha, beta, X, Y = create_data(M=20, N=500, r_mean=0.8, r_std=0.1)
intercept, coefficients = build_model(X, Y)

p_bar = np.full(20, 0.8)  # 平均価格ベクトルp̄をr_meanで初期化
epsilon = 0.09 * np.linalg.norm(coefficients)  # 許容誤差ε

In [10]:
# sdpの解法を呼び出す
new_prices_sdp, new_max_revenue_sdp, new_solve_time = optimize_price_sdp_mosek(
    intercept,
    coefficients,
    p_bar=p_bar,
    epsilon=epsilon
)

--- SDP Dual Formulation with Price Recovery (MOSEK) ---
Optimal Prices: [1.255  1.1617 1.252  1.1534 1.0132 1.2184 1.085  1.1301 0.9682 0.9505
 1.0194 0.7664 0.5824 1.1343 1.437  1.0511 1.0118 0.9247 1.0676 1.0696]
Maximum Revenue: 376.4111
Solve Time: 0.001983 seconds


In [11]:
new_prices_mosek, new_max_revenue_mosek, new_solve_time_mosek = optimize_price_new_mosek(
    intercept,
    coefficients,
    p_bar=p_bar,
    epsilon=epsilon
)

In [12]:
print("SDP解法の結果:"
      f"\n新しい価格: {new_prices_sdp}\n"
      f"最大収益: {new_max_revenue_sdp}\n"
      f"解法時間: {new_solve_time:.4f}秒"
      )
print("Mosek解法の結果:"
      f"\n新しい価格: {new_prices_mosek}\n"
      f"最大収益: {new_max_revenue_mosek}\n"
      f"解法時間: {new_solve_time_mosek:.4f}秒"
      )

SDP解法の結果:
新しい価格: [1.25497414 1.16167285 1.25200444 1.15343344 1.0131568  1.21839668
 1.08498248 1.13010105 0.96822979 0.95046606 1.01937099 0.76637565
 0.58235943 1.13430425 1.43699356 1.05107215 1.01180067 0.92470617
 1.06755706 1.06963047]
最大収益: 376.41108360989523
解法時間: 0.0020秒
Mosek解法の結果:
新しい価格: [1.29573799 1.08342926 1.34818875 1.09696268 1.03860427 1.1393268
 1.20511486 1.08029242 0.98256305 0.95553693 0.98571116 0.78807635
 0.75074206 1.1959692  1.30317007 1.00422383 1.06298292 0.85538627
 1.07935017 1.09135862]
最大収益: 376.4110836362738
解法時間: 0.0012秒


In [13]:
print(calculate_true_revenue(
    optimal_prices=new_prices_mosek,
    alpha_true=alpha,
    beta_true=beta
))

(1232.9960578340324, array([72.4828973 , 74.75267428, 50.70825188, 90.56129801, 85.58250985,
       75.843731  , 47.19776853, 35.52686049, 33.72228146, 34.29541169,
       41.53067736, 35.78024452, 37.0024603 , 61.89068018, 68.94141528,
       69.29318119, 84.14287451, 37.38437547, 57.1374947 , 37.0800483 ]))


In [14]:
print(calculate_true_revenue(
    optimal_prices=new_prices_sdp,
    alpha_true=alpha,
    beta_true=beta
))

(1242.2057812085177, array([71.27625765, 79.1325276 , 47.08583405, 93.75357833, 84.77209315,
       80.40993682, 42.10751604, 37.82724435, 33.61969422, 34.405388  ,
       43.68659575, 34.70538369, 27.87548104, 59.28476854, 74.83704133,
       72.14749372, 80.9642309 , 40.80480606, 56.13241129, 36.11880557]))


In [15]:
print(new_solve_time)

0.016515016555786133


In [16]:
M_g1, M_g2 = create_sdp_matrices(intercept, coefficients, p_bar, epsilon)
mu_1, mu_2 , solve_time = solve_sdp_with_gurobi(M_g1, M_g2, np.eye(len(p_bar) + 1))

Set parameter Username
Set parameter LicenseID to value 2659505
Academic license - for non-commercial use only - expires 2026-04-30


AttributeError: type object 'GRB' has no attribute 'SDP'

In [ ]:
gp.gurobi.version()


(12, 0, 2)

In [53]:
import cvxpy as cp
import numpy as np

def solve_sdp_via_cvxpy(M_g1, M_g2, M_I):
    n = M_g1.shape[0]

    # 変数定義
    X   = cp.Variable((n, n), PSD=True)   # PSD=True で半正定値制約
    mu1 = cp.Variable(nonneg=True)
    mu2 = cp.Variable()

    # 制約
    cons = [
        X == M_g2 + mu1 * M_g1 - mu2 * M_I
    ]

    # 目的はmu2を最大化（元問題では -mu2 の最小化と同義）
    prob = cp.Problem(cp.Maximize(mu2), cons)

    # Gurobi を内部 SDP ソルバーとして指定
    prob.solve(solver=cp.GUROBI, verbose=True)

    if prob.status == cp.OPTIMAL:
        print(f"Optimal mu1 = {mu1.value:.4f}")
        print(f"Max revenue = {-mu2.value:.4f}")
        return mu1.value, mu2.value, prob.solver_stats.time
    else:
        print("Failed to solve")
        return None, None, None


In [54]:
M_g1, M_g2 = create_sdp_matrices(intercept, coefficients, p_bar, epsilon)
mu_1, mu_2 , solve_time = solve_sdp_via_cvxpy(M_g1, M_g2, np.eye(len(p_bar) + 1))

(CVXPY) Jul 07 06:19:27 PM: Your problem has 443 variables, 441 constraints, and 0 parameters.
(CVXPY) Jul 07 06:19:27 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 07 06:19:27 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 07 06:19:27 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jul 07 06:19:27 PM: Your problem is compiled with the CPP canonicalization backend.


                                     CVXPY                                     
                                     v1.6.6                                    


SolverError: Either candidate conic solvers (['GUROBI']) do not support the cones output by the problem (Zero, PSD), or there are not enough constraints in the problem.